In [2]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("GPU Count:", torch.cuda.device_count())


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
GPU Count: 1


In [4]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import cv2
import os
import timm  # For EfficientNetV2
from ultralytics import YOLO
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
import torch_pruning as tp

In [5]:
import cv2
import numpy as np

def extract_spectral_features(image):
    spectral_info = image[:, :, 0] - image[:, :, 2]  # Red - Blue
    return spectral_info

def extract_spatial_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kernel = np.ones((3, 3), np.float32) / 9
    background_avg = cv2.filter2D(gray_image, -1, kernel)
    spatial_info = gray_image - background_avg
    return spatial_info

def extract_temporal_features(current_image, previous_image=None):
    current_gray = cv2.cvtColor(current_image, cv2.COLOR_RGB2GRAY)
    if previous_image is None:
        return np.zeros_like(current_gray)
    previous_gray = cv2.cvtColor(previous_image, cv2.COLOR_RGB2GRAY)
    temporal_info = current_gray - previous_gray
    return temporal_info

def extract_and_fuse_features(image, previous_image=None):
    spectral_features = extract_spectral_features(image)
    spatial_features = extract_spatial_features(image)
    temporal_features = extract_temporal_features(image, previous_image)
    
    # Apply weighted sum
    weights = [0.33, 0.33, 0.34]  # Adjust weights based on feature importance
    fused_features = weights[0] * spectral_features + weights[1] * spatial_features + weights[2] * temporal_features
    
    return fused_features

# Example usage
image = cv2.imread(r"C:\Users\lohit\OneDrive\Downloads\fire_at_kiphire.jpg")
previous_image = None  # Or provide a previous image for temporal features

fused_features = extract_and_fuse_features(image, previous_image)

print(f"Fused Features Shape: {fused_features.shape}")


Fused Features Shape: (962, 1388)


In [8]:
import os

# Define the datasets directory in the home folder
datasets_path = os.path.join(os.path.expanduser("~"), "datasetsrev6")

# Create the directory if it doesn't exist
os.makedirs(datasets_path, exist_ok=True)

# Change directory to datasets folder
os.chdir(datasets_path)

# Verify current working directory
print("Current Directory:", os.getcwd())

# Install roboflow
!pip install roboflow


Current Directory: C:\Users\lohit\datasetsrev6


In [10]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="d9mYKhk9HLFYK6BfTozC")
project = rf.workspace("lohit-p9kuf").project("fire-smoke-and-human-detector-1p9yk")
version = project.version(1)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...


In [12]:
import os

# Define the dataset path
dataset_path = os.path.join(os.getcwd(), "Fire-Smoke-and-Human-Detector-1")

# List the files in the dataset directory
print("Dataset files:", os.listdir(dataset_path))

Dataset files: ['data.yaml', 'README.dataset.txt', 'README.roboflow.txt', 'test', 'train', 'valid']


In [16]:
import os

# Get the user's home directory
home_dir = os.path.expanduser("~")

# Define the datasets directory path
datasets_path = os.path.join(home_dir, "datasetsrev6")

# Create the directory if it doesn't exist
os.makedirs(datasets_path, exist_ok=True)

# Change to the datasets directory
os.chdir(datasets_path)

# Verify current working directory
print("Current Directory:", os.getcwd())

# Install required libraries
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="d9mYKhk9HLFYK6BfTozC")
project = rf.workspace("lohit-p9kuf").project("fire-smoke-and-human-detector-1p9yk")
version = project.version(1)
dataset = version.download("yolov12")
                

# Define the dataset path
dataset_path = os.path.join(os.getcwd(), "Fire-Smoke-and-Human-Detector-1")

# List the files in the dataset directory
if os.path.exists(dataset_path):
    print("Dataset files:", os.listdir(dataset_path))
else:
    print("Dataset directory not found:", dataset_path)

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8s.pt")

# Train YOLO normally first
data_yaml_path = os.path.join(dataset_path, "data.yaml")

if os.path.exists(data_yaml_path):
    model.train(data=data_yaml_path, 
                epochs=100, imgsz=640, batch=16)
else:
    print("data.yaml file not found:", data_yaml_path)


Current Directory: C:\Users\lohit\datasetsrev6
loading Roboflow workspace...
loading Roboflow project...
Dataset files: ['data.yaml', 'README.dataset.txt', 'README.roboflow.txt', 'test', 'train', 'valid']


100%|██████████| 21.5M/21.5M [00:00<00:00, 36.5MB/s]


New https://pypi.org/project/ultralytics/8.3.97 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\lohit\datasetsrev6\Fire-Smoke-and-Human-Detector-1\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cla

train: Scanning C:\Users\lohit\datasetsrev6\Fire-Smoke-and-Human-Detector-1\train\labels.cache... 24003 images, 2457 backgrounds, 0 corrupt: 100%|██████████| 24003/24003 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 36, len(boxes) = 59681. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    100/100      3.61G     0.7311      0.434      1.108          7        640: 100%|██████████| 1501/1501 [08:08<00:00,  3.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:40<00:00,  1.26s/it]

                   all       1017       2882      0.871      0.799      0.836      0.685



100 epochs completed in 14.945 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 22.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.96  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:10<00:00,  3.09it/s]


                   all       1017       2882       0.87      0.811      0.843      0.686
                  fire        905       1627      0.975      0.978      0.988       0.95
                 human        220        455      0.709      0.648      0.669      0.303
                 smoke        482        800      0.926      0.805      0.872      0.805
Speed: 0.3ms preprocess, 5.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train2
